In [1]:
import obspy, glob
import os, glob
import numpy as np
import pandas as pd
import numpy.matlib
from obspy import Trace, UTCDateTime, Stream
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib import cm
from scipy import signal, ndimage
from scipy.io.wavfile import read, write
import cv2
import dask
import dask_image.ndfilters
import dask.array
from datetime import datetime, date, timedelta
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

In [2]:
path = '/Volumes/Data/Soundscape/03_TaoYuan/2023/20230818-20230902/wav/'
place = path.rsplit('/')[4].rsplit('_')[1]


S = -176.1 # end to end 
P_ref = 1 # water: 1 uPa
interval = 10 # sec
fmin = 100 # Hz
fmax = 1000 # Hz

# local_cluster = LocalCluster(n_workers=2, processes=False, threads_per_worker=16)

In [3]:
def calib(data, S):
    audio_data = data.astype(np.float32, order='C') / 32768.0
    calib_data = audio_data / ((10)**(S/20))
    return calib_data # uPa

          
def spectrogram(data, sampling_frequency, window='hann', fmin=200, fmax=1000, verbose=False):
    if verbose:
        print('--------------- Calculate spectro.')
    f, t, spect = signal.spectrogram(
                    data,
                    fs=sampling_frequency,
                    window=window,
                    nperseg=int(sampling_frequency/20),
                    noverlap=int(sampling_frequency/20*0.9),
                    nfft=2**16,
                    scaling="spectrum" )
    
    fmin_idx = np.where(f <= fmin)[0][-1]+1
    fmax_idx = np.where(f <= fmax)[0][-1]-1
    f = f[fmin_idx:fmax_idx]
    spect = spect[fmin_idx:fmax_idx, :]
    return f, t, spect

def denoise(spect, duration=10, time_resolution=0.005, use_dask=True, verbose=False):
    if use_dask:
        
        dask_spectro = dask.array.from_array(spect, chunks=[1000,1000])
        Smed = dask_image.ndfilters.percentile_filter(
            dask_spectro, 50,
            size=(1, round(duration / time_resolution)),
            mode="mirror",
        )
        Smed = Smed.compute()
    else:
        
        Smed = ndimage.percentile_filter(
                spect, 50,
                (1, round(duration / time_resolution)) ) 
    
    spect = spect - Smed
    spect[spect < 0] = 0  # floor
    
    return spect
def detect(t, f, spect, tr, starttime, kernel_duration=0.05, kernel_bandwidth=300, threshold_pct=90, f_noise=None, use_dask=False, verbose=False):
    if verbose:
        print('--------------- Calculate detection.  ')
    f_res = f[1]-f[0]
    t_res = round(t[1]-t[0], 5)
    kernel_duration = max(round(kernel_duration/t_res), 1) #sec to pt
    kernel_bandwidth = max(round(kernel_bandwidth/f_res), 1) #Hz to pt
    if use_dask:
        dask_spectro = dask.array.from_array(spect, chunks=[1000,1000])
        Svar = dask_image.ndfilters.generic_filter(dask_spectro,
                                                   calcVariance2D,
                                                   size=(kernel_bandwidth, kernel_duration),
                                                   mode='mirror')
        Svar = Svar.compute()
    else:
        imx = np.zeros(spect.shape)
        Svar = ndimage.generic_filter(spect,
                                  calcVariance2D,
                                  (kernel_bandwidth, kernel_duration),
                                  mode='mirror')
        # Svar = ndimage.generic_filter(spect,
        #                           calcVariance2D,
        #                           (kernel_bandwidth, kernel_duration),
        #                           mode='mirror')
        
    # binarization
    threshold = np.percentile(Svar, threshold_pct, interpolation='linear')
    Svar[Svar < threshold] = 0
    Svar[Svar > 0] = 1
    
    
    Svar_gray = cv2.normalize(src=Svar,
                              dst=None,
                              alpha=0,
                              beta=255,
                              norm_type=cv2.NORM_MINMAX,
                              dtype=cv2.CV_8UC1)
    (cnts, hierarchy) = cv2.findContours(Svar_gray.copy(),
                                              cv2.RETR_EXTERNAL,
                                              cv2.CHAIN_APPROX_SIMPLE)   
    cnts_sorted = sorted(list(cnts),key=lambda d: d[0][0][0])
    # loop over the contours
    isdetec = False
    t1 = []; t2 = []
    fmin = []; fmax = []
    
    for ii, c in enumerate(cnts_sorted):
        # Compute the bounding box for the contour
        (x, y, w, h) = cv2.boundingRect(c)
        if w < kernel_duration or h < kernel_bandwidth: # if the contour is too small, ignore it
            continue
        elif h >= round((f[-1]-f[0])/f_res, 2)*0.85 and w*t_res < 0.02: ## avoid short and broadband-noise 
            #print('broadband-noise', w)
            continue
        elif f_noise is not None:
            peak_f_detect = Peak_frequency(f[y:y+h-1], spect[y:y+h-1, x:x+w-1]) 
            if abs(peak_f_detect-f_noise) > 20 : ## avoid continous noise
                t1.append(x); t2.append(x+w-1) 
                fmin.append(y); fmax.append(y+h-1) 
            else:
                #print(peak_f_detect,' low-frequency noise')
                pass
        else:
            isdetec = True

            if len(t1)>0 and t2[-1]>=x>=t1[-1]:    ## avoid repeated detection
                _fmin = fmin[-1]; _fmax = fmax[-1]
                if y<_fmin :
                    fmin[-1] = y
                if (y+h-1)>_fmax:
                    fmax[-1] = y+h-1
                continue
            else:
                t1.append(x); t2.append(x+w-1) # box coord
                fmin.append(y); fmax.append(y+h-1)   
    
    detec = {} 
    detec['time_min_offset'] = [t[0]+_t*t_res for _t in t1]
    detec['time_max_offset'] = [t[0]+_t*t_res for _t in t2]
    detec['frequency_min'] = [f[_f] for _f in fmin]
    detec['frequency_max'] = [f[_f] for _f in fmax]
    detec['duration'] = np.array(detec['time_max_offset']) - np.array(detec['time_min_offset'])
    detec['time_min_date'] = [starttime+_t*t_res+t[0] for _t in t1] 
    detec['time_max_date'] = [starttime+_t*t_res+t[0] for _t in t2]
    detec['SPLpp'] = [ SPL_PP(tr, t1, t2) for t1, t2 in zip(detec['time_min_date'], detec['time_max_date'])]
    detec = pd.DataFrame(detec)
    detec.sort_values('time_min_offset',
                       ascending=True,
                       inplace=True,
                       ignore_index=True )
    return Svar, detec, cnts_sorted

def calcVariance2D(buffer):
    """Calculate the 2D variance."""
    return np.var(buffer)
    # return np.median(buffer.ravel())
    # return np.mean(buffer.ravel())
    
def plot_annot(detectfile, ax, type='spectrogram', color='red'):
    if type == 'spectrogram':
        alpha = 0.7
        facecolor = "none"
        color = color
        label='detected'
        line_width = 0.7
        for index, row in detectfile.iterrows():
            x = row["time_min_offset"]
            y = row["frequency_min"]
            width = row["duration"]
            height = row["frequency_max"] - row["frequency_min"]
            import matplotlib.patches as patches
            rect = patches.Rectangle((x, y), width, height, linewidth=line_width, edgecolor=color, facecolor=facecolor, alpha=alpha)
            ax.add_patch(rect)  
    elif type == 'waveform':
        alpha = 0.1
        facecolor = color
        color='none'
        label='detected'
        line_width = 0.5
        for index, row in detectfile.iterrows():
            x = row["time_min_offset"]
            y = min(ax.get_ylim())
            width = row["duration"]
            height = max(ax.get_ylim()) - min(ax.get_ylim())
            
            import matplotlib.patches as patches
            rect = patches.Rectangle((x, y), width, height, linewidth=line_width, edgecolor=color, facecolor=facecolor, alpha=alpha)
            ax.add_patch(rect)  
    return ax

def SPL_PP(tr, t1, t2):
    P_ref = 1
    tr_sound = tr.slice(t1, t2)
    p_peak = max(tr_sound.data)+abs(min(tr_sound.data))
    P_spl = 20*np.log10(p_peak/P_ref)
    return P_spl

def continuous_singal(f, data):
    dev_f_peak = []
    t_shape = data.shape[1]
    for i in range(0, t_shape, 400):
        data_tmp = data[:, i:i+400]
        Emax_freq = Peak_frequency(f, data_tmp)
        if i != 0:
            if abs(previous_Emax_freq-Emax_freq) <= 20:
                dev_f_peak.append(True)
            else:
                dev_f_peak.append(False)
            previous_Emax_freq = Emax_freq
        else:
            previous_Emax_freq = Emax_freq
    if dev_f_peak[:3] == [True]*3 or dev_f_peak[1:] == [True]*3:
        Emax_freq = Peak_frequency(f, data)
        return True, Emax_freq
    else:
        return False, None  
    
    
def Peak_frequency(f, data):
    sum_energy = np.sum(data, axis=1)
    Emax_freq = f[np.argmax(sum_energy)]
    return Emax_freq

In [4]:
df_min = {'Time':[],
         'Peak': [],
         'MaxSPL': [], 
         'SPLrms':[]}
for file in glob.glob(f'{path}/????.23082005*.wav'):
    print(file)
    rate, data_ = read(file) 
    data = calib(data_, S)
    
    filetime = file.rsplit('.')[1]
    tr = Trace()
    tr.data = data
    tr.stats.sampling_rate = rate
    if rate == 96000:
        tr.decimate(2, no_filter=True) 
    tr.stats.starttime = UTCDateTime(2023, int(filetime[2:4]), int(filetime[4:6]), int(filetime[6:8]), int(filetime[8:10]), int(filetime[10:]))
    tr_bp = tr.copy()
    tr_bp.filter('bandpass', freqmin=fmin, freqmax=fmax, corners=6, zerophase=True)
    
    total_min = int(tr.stats.endtime - tr.stats.starttime)/60
    

#     for mm in range(int(total_min)):

print('finished ')

/Volumes/Data/Soundscape/03_TaoYuan/2023/20230818-20230902/wav/6979.230820050001.wav
finished 


## for plotting 

In [5]:
alldetect = pd.DataFrame([])
plot = True
save = True

verbose = False
if save:
    savepath = '/Volumes/home/Research/Detect_fish/Taoyuan'
    dirtime = file.rsplit('.')[1][:8]
    if not os.path.isdir(f'{savepath}_csv/{dirtime}'):
        os.mkdir(f'{savepath}_csv/{dirtime}')
        os.mkdir(f'{savepath}_fig/{dirtime}')
    
    

t0 = datetime.now()
total_sec = int(tr_bp.stats.endtime - tr_bp.stats.starttime)
for dt in np.arange(2370, total_sec, interval):
    print(dt)
    tr_cut = tr_bp.copy() 
    start  = tr_bp.stats.starttime + dt 
    end    = start + interval
    tr_cut = tr_cut.slice(starttime=start, endtime=end)
    tr_cut.data = tr_cut.data - np.median(tr_cut.data)

    
    f, t, spect = spectrogram(tr_cut.data, tr_cut.stats.sampling_rate,  fmin=fmin, fmax=fmax, verbose=verbose)
    
    spect_denoise = denoise(spect, verbose=verbose)
    
    cont_noise, freq_noise = continuous_singal(f, spect_denoise)
    Svar_, detec, cnts_sorted = detect(t, f, spect_denoise, tr_cut, starttime=tr_cut.stats.starttime, use_dask=True, \
                                       threshold_pct=90, f_noise=freq_noise, kernel_duration=0.05, kernel_bandwidth=300, verbose=verbose)
    
    var_tr_cut = np.var(tr_cut.data)
    snrlist = []
    for ii in range(len(detec)):
        dect_start = detec['time_min_date'].values[ii]
        dect_end = detec['time_max_date'].values[ii]
        
        tr_dect = tr_cut.slice(dect_start, dect_end)
        var_tr_dect = np.var(tr_dect.data)
        snr = 10*np.log10(var_tr_dect/var_tr_cut)
        snrlist.append(snr)
            
    new_detec = pd.concat([detec, 
                           pd.DataFrame({'low_freq':[freq_noise]*len(detec), 'SNR': snrlist})], axis=1)
    # alldetect = pd.concat([alldetect, new_detec])
    
    if plot:
        fig, ax = plt.subplots(3, 1 ,figsize=(7, 5), sharex=True)
        plt.rc('font', family='Helvetica')
        ax[0].plot(tr_cut.times(), tr_cut.data, 'k')
        ax[0].set_xlim(tr_cut.times()[0], tr_cut.times()[-1])
        ax[0].set_ylabel('Pressure ($\mu$Pa)')
        

        ax[1].pcolormesh(t, f, spect_denoise, cmap="Blues",
                        vmin=np.percentile(spect_denoise, 90),
                        vmax=np.percentile(spect_denoise, 99.9),
                        shading="gouraud")
        ax[1].set_ylim(fmin, fmax)
        ax[1].set_ylabel('Frequency (Hz)')

        ax[2].pcolormesh(t, f, Svar_, cmap="binary")
        ax[2].set_ylim(fmin, fmax)
        ax[2].set_xlabel('Time (s)')
        ax[2].set_ylabel('Frequency (Hz)')

        high_SNR = new_detec[new_detec['SNR'] >= 3]
        low_SNR = new_detec[new_detec['SNR']  < 3.0]
        ax[1] = plot_annot(high_SNR, ax[1], type='spectrogram', color='red')
        ax[0] = plot_annot(high_SNR, ax[0], type='waveform', color='red')
        ax[1] = plot_annot(low_SNR, ax[1], type='spectrogram', color='blue')
        ax[0] = plot_annot(low_SNR, ax[0], type='waveform', color='blue')
        if freq_noise is None:
            title = f'# of high SNR detection: {len(high_SNR)} low SNR: {len(low_SNR)}\n\n{place} {tr_cut.stats.starttime}'
        else:
            title = f'# of high SNR detection: {len(high_SNR)} low SNR: {len(low_SNR)}\nPeak frequency: {int(freq_noise)} Hz\n{place} {tr_cut.stats.starttime}'
        ax[0].set_title(title)
        if save:
            fig.savefig(f'{savepath}_fig/{dirtime}/{place}_{file.rsplit(".")[1]}_{dt}.png', dpi=150, transparent=True)
            plt.close()
            
    if save:
        new_detec.to_csv(f'{savepath}_csv/{dirtime}/{place}_{file.rsplit(".")[1]}_{dt}.csv', index=False)
    # del Svar_, detec, cnts_sorted, spect_denoise, spect
        
t1 = datetime.now()
print(f'calculating time {t1-t0}')
# fig.savefig('Example2.png', dpi = 200)

2370
2380
2390
2400
2410
2420
2430
2440
2450
2460
2470
2480
2490
2500
2510
2520
2530
2540
2550
2560
2570
2580
2590
2600
2610
2620
2630
2640
2650
2660
2670
2680
2690
2700
2710
2720
2730
2740
2750
2760
2770
2780
2790
2800
2810
2820
2830
2840
2850
2860
2870
2880
2890
2900
2910
2920
2930
2940
2950
2960
2970
2980
2990
3000
3010
3020
3030
3040
3050
3060
3070
3080
3090
3100
3110
3120
3130
3140
3150
3160
3170
3180
3190
3200
3210
3220
3230
3240
3250
3260
3270
3280
3290
3300
3310
3320
3330
3340
3350
3360
3370
3380
3390
3400
3410
3420
3430
3440
3450
3460
3470
3480
3490
3500
3510
3520
3530
3540
3550
3560
3570
3580
3590
calculating time 3:06:10.073506


## to listen 

In [ ]:
for file in glob.glob(f'{path}/????.23062512*.wav'):
    print(file)
    rate, data_ = read(file) 
    data = calib(data_, S)
    
    filetime = file.rsplit('.')[1]
    tr = Trace()
    tr.data = data
    tr.stats.sampling_rate = rate
    if rate == 96000:
        tr.decimate(2, no_filter=True) 
    tr.stats.starttime = UTCDateTime(2023, int(filetime[2:4]), int(filetime[4:6]), int(filetime[6:8]), int(filetime[8:10]), int(filetime[10:]))
    tr_bp = tr.copy()
    tr_bp.filter('bandpass', freqmin=fmin, freqmax=fmax, corners=6, zerophase=True)
    
filename = f"../Results/Detection_fishcall/99_tolisten/{place}_{file.rsplit('.')[1]}_downsampling_bp"
write(f'{filename}.wav', int(tr_bp.stats.sampling_rate), np.int16(tr_bp.data/np.max(np.abs(tr_bp.data)) * 32767))

print(f'finished {filename}')

In [12]:
round(t[1]-t[0], 5)

0.005

In [17]:
20*0.005

0.1

In [18]:
0.002/0.005

0.4